<a href="https://colab.research.google.com/github/JarupongOfficial/maskDetection/blob/main/trainModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("Initial Google Collab to Github")

Initial Google Collab to Github


# Mask Detection with Yolov5


### Setup system environment for Yolov5 and Python package

In [3]:
!ls

drive  sample_data


In [4]:
#Download yolov5 from Github
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 9008, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 9008 (delta 21), reused 26 (delta 15), pack-reused 8963
Receiving objects: 100% (9008/9008), 9.47 MiB | 25.38 MiB/s, done.
Resolving deltas: 100% (6265/6265), done.


In [5]:
#Install required python packages and weight&bias
%cd /content/yolov5
!ls
!pip install -r requirements.txt
!pip install wandb

/content/yolov5
CONTRIBUTING.md  Dockerfile  LICENSE	requirements.txt  utils
data		 export.py   models	train.py	  val.py
detect.py	 hubconf.py  README.md	tutorial.ipynb
     |████████████████████████████████| 636 kB 11.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.6 MB 13.2 MB/s 
     |████████████████████████████████| 133 kB 51.6 MB/s 
     |████████████████████████████████| 170 kB 39.1 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=4701482cd5482a013c83b40edfa5bea3acd5e0fe7d97c381c7a7b982cb02bc71
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.w

In [6]:
#Test with pytorch and GPU/GPU hardware
import torch
print(f"Check pytorch version: {torch.__version__}")
print(f"Check pytorch hardware: {torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'}")

Check pytorch version: 1.9.0+cu102
Check pytorch hardware: Tesla K80


In [7]:
#Login to wegiht and bias system
!pip install -q --upgrade wandb
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
#Import python package/ library
import numpy as np
import pandas as pd

import os
from shutil import copyfile
from pathlib import Path
from xml.dom.minidom import parse

from sklearn.model_selection import train_test_split

### Preparing annotation files from PascalVOC to yolov5 format

In [ ]:
for dirname, _, filenames in os.walk('/content/sample_data'):
  print("Hello")
  for filename in filenames:
    print(os.path.join(dirname, filename))

Hello
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/mnist_train_small.csv
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/california_housing_train.csv


In [ ]:
#Prepare directory for Yolov5 format
%cd ..
!mkdir -p Dataset/labels
!mkdir -p Dataset/images

/content


In [ ]:
#Assign up variable related to each datapath in string format
# Path('drive/MyDrive/PascalVOC').is_dir()
ORIGINAL_ROOT = "/content/drive/MyDrive/PascalVOC"
O_IMG_ROOT = ORIGINAL_ROOT + "/images"
O_LABEL_ROOT = ORIGINAL_ROOT+"/annotations"

MODIFIED_ROOT = "/content/Dataset"
M_IMG_ROOT = MODIFIED_ROOT+"/images"
M_LABEL_ROOT = MODIFIED_ROOT+"/labels"

In [ ]:
classes = ['with_mask','without_mask','mask_weared_incorrect']

In [ ]:
#This function will transform PasvalVOC to Yolov5 format
#by parse input and calculate to BB algorithm.
def convertFromXML(parsed_img_size, box):
    x1=int(box[0])
    y1=int(box[1])
    x2=int(box[2])
    y2=int(box[3])

    dw = np.float32(1. / int(parsed_img_size[0]) )
    dh = np.float32(1. / int(parsed_img_size[1]) )

    w = x2 - x1
    h = y2 -y1
    x = x1 + (w/2)
    y = y1 + (h/2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h *dh

    return [x, y, w, h]

In [ ]:
#This function will create new annotation file in Yolov5 format
def save_txt_file(img_png_file_name, parsed_img_size, label_box):
    save_file_name = M_LABEL_ROOT + '/' + img_png_file_name +'.txt'
    print(save_file_name)

    with open(save_file_name, 'a+') as file_path:
        for box in label_box:

            class_num = classes.index(box[0])

            yolov5_box = convertFromXML(parsed_img_size, box[1:])

            file_path.write(f"{class_num} {yolov5_box[0]} {yolov5_box[1]} {yolov5_box[2]} {yolov5_box[3]}\n")


        file_path.flush()
        file_path.close()

In [ ]:
#This function will access and parsing on original PasvalVOC format 
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data

    label_box = []
    for box in objects:
        class_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = box.getElementsByTagName("xmin")[0].childNodes[0].data
        y1 = box.getElementsByTagName("ymin")[0].childNodes[0].data
        x2 = box.getElementsByTagName("xmax")[0].childNodes[0].data
        y2 = box.getElementsByTagName("ymax")[0].childNodes[0].data

        img_png_file_name = img_xml_file + '.png'
        label_box.append([class_name, x1, y1, x2, y2])

    save_txt_file(img_xml_file, [img_w, img_h], label_box)

In [ ]:
files = os.listdir(O_LABEL_ROOT)
for file in files:
    # print("file name:", file)
    file_xml = file.split(".")
    get_xml_data(O_LABEL_ROOT, file_xml[0])

/content/Dataset/labels/maksssksksss105.txt
/content/Dataset/labels/maksssksksss1.txt
/content/Dataset/labels/maksssksksss104.txt
/content/Dataset/labels/maksssksksss11.txt
/content/Dataset/labels/maksssksksss10.txt
/content/Dataset/labels/maksssksksss101.txt
/content/Dataset/labels/maksssksksss103.txt
/content/Dataset/labels/maksssksksss106.txt
/content/Dataset/labels/maksssksksss110.txt
/content/Dataset/labels/maksssksksss102.txt
/content/Dataset/labels/maksssksksss107.txt
/content/Dataset/labels/maksssksksss108.txt
/content/Dataset/labels/maksssksksss100.txt
/content/Dataset/labels/maksssksksss0.txt
/content/Dataset/labels/maksssksksss109.txt
/content/Dataset/labels/maksssksksss33.txt
/content/Dataset/labels/maksssksksss76.txt
/content/Dataset/labels/maksssksksss187.txt
/content/Dataset/labels/maksssksksss417.txt
/content/Dataset/labels/maksssksksss150.txt
/content/Dataset/labels/maksssksksss676.txt
/content/Dataset/labels/maksssksksss825.txt
/content/Dataset/labels/maksssksksss394.

In [ ]:
!zip -r /content/label.zip /content/Dataset/

  adding: content/Dataset/ (stored 0%)
  adding: content/Dataset/labels/ (stored 0%)
  adding: content/Dataset/labels/maksssksksss419.txt (deflated 57%)
  adding: content/Dataset/labels/maksssksksss616.txt (deflated 63%)
  adding: content/Dataset/labels/maksssksksss826.txt (deflated 57%)
  adding: content/Dataset/labels/maksssksksss249.txt (deflated 46%)
  adding: content/Dataset/labels/maksssksksss521.txt (deflated 28%)
  adding: content/Dataset/labels/maksssksksss141.txt (deflated 42%)
  adding: content/Dataset/labels/maksssksksss433.txt (deflated 28%)
  adding: content/Dataset/labels/maksssksksss464.txt (deflated 52%)
  adding: content/Dataset/labels/maksssksksss143.txt (deflated 28%)
  adding: content/Dataset/labels/maksssksksss703.txt (deflated 57%)
  adding: content/Dataset/labels/maksssksksss783.txt (deflated 52%)
  adding: content/Dataset/labels/maksssksksss665.txt (deflated 54%)
  adding: content/Dataset/labels/maksssksksss408.txt (deflated 60%)
  adding: content/Dataset/label

In [ ]:
from google.colab import files
files.download("/content/label.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Import sklearn to split images in data and keep in list
from sklearn.model_selection import train_test_split
image_list = os.listdir(O_IMG_ROOT)
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=42)
validate_list, test_list = train_test_split(test_list, test_size=0.5, random_state=42)
print('Total Images: ', len(image_list))
print('Train images: ', len(train_list))
print('Test images:', len(test_list))
print('Validate images', len(validate_list))

Total Images:  853
Train images:  682
Test images: 86
Validate images 85


In [ ]:
#Transform previous list to split and move real image and labels files
def split_model_image(file_list, img_labels_root, img_source,  mode):

    image_path = Path(M_IMG_ROOT+'/'+ mode)
    if not image_path.exists():
        print(f"path {image_path} does not exist")
        os.makedirs(image_path)

    label_path = Path(M_LABEL_ROOT+'/'+ mode)
    if not label_path.exists():
        print(f"Path {label_path} does not exist")
        os.makedirs(label_path)

    for file in file_list:
        img_name = file.replace('.png', '')
        img_src_file = img_source+'/'+img_name+'.png'
        label_src_file = img_labels_root+'/'+img_name+'.txt'

        IMG_DICT_DIR = M_IMG_ROOT+'/'+mode
        img_dict_file = IMG_DICT_DIR+'/'+img_name+'.png'
        copyfile(img_src_file, img_dict_file)

        LABEL_DICT_DIR = M_LABEL_ROOT+'/'+mode
        label_dict_file = LABEL_DICT_DIR+'/'+img_name+'.txt'
        copyfile(label_src_file,label_dict_file)

In [ ]:
split_model_image(train_list, M_LABEL_ROOT+'/', O_IMG_ROOT,"train")
split_model_image(test_list, M_LABEL_ROOT+'/', O_IMG_ROOT,"test")
split_model_image(validate_list, M_LABEL_ROOT+'/', O_IMG_ROOT,"validate")

path /content/Dataset/images/train does not exist
Path /content/Dataset/labels/train does not exist
path /content/Dataset/images/test does not exist
Path /content/Dataset/labels/test does not exist
path /content/Dataset/images/validate does not exist
Path /content/Dataset/labels/validate does not exist


In [ ]:
!zip -r /content/image.zip /content/Dataset/

  adding: content/Dataset/ (stored 0%)
  adding: content/Dataset/labels/ (stored 0%)
  adding: content/Dataset/labels/maksssksksss419.txt (deflated 57%)
  adding: content/Dataset/labels/maksssksksss616.txt (deflated 63%)
  adding: content/Dataset/labels/maksssksksss826.txt (deflated 57%)
  adding: content/Dataset/labels/maksssksksss249.txt (deflated 46%)
  adding: content/Dataset/labels/maksssksksss521.txt (deflated 28%)
  adding: content/Dataset/labels/maksssksksss141.txt (deflated 42%)
  adding: content/Dataset/labels/maksssksksss433.txt (deflated 28%)
  adding: content/Dataset/labels/maksssksksss464.txt (deflated 52%)
  adding: content/Dataset/labels/maksssksksss143.txt (deflated 28%)
  adding: content/Dataset/labels/maksssksksss703.txt (deflated 57%)
  adding: content/Dataset/labels/maksssksksss783.txt (deflated 52%)
  adding: content/Dataset/labels/maksssksksss665.txt (deflated 54%)
  adding: content/Dataset/labels/maksssksksss408.txt (deflated 60%)
  adding: content/Dataset/label

In [ ]:
from google.colab import files
files.download("/content/image.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Training Yolov5 Model with Prepared Dataset

In [9]:
#Assign up variable related to each datapath in string format
# Path('drive/MyDrive/PascalVOC').is_dir()
ORIGINAL_ROOT = "/content/drive/MyDrive/PascalVOC"
O_IMG_ROOT = ORIGINAL_ROOT + "/images"
O_LABEL_ROOT = ORIGINAL_ROOT+"/annotations"

MODIFIED_ROOT = "/content/drive/MyDrive/Dataset"
M_IMG_ROOT = MODIFIED_ROOT+"/images"
M_LABEL_ROOT = MODIFIED_ROOT+"/labels"